# Session 2 — Prompt Engineering 실습 (V1)

이 노트북은 *고급 프롬프트 엔지니어링* 실습용으로 준비되었습니다.  
- **문제 정의 → V0 → V1 → V2** 순으로 단계별 개선 과정을 체험합니다.  
- 오직 **`gpt‑4o‑mini`** 모델만 사용합니다.  
- 실제 회사 데이터 대신 *샘플* 시나리오 3종(`order_delivery`, `refund`, `account_login`)을 사용합니다.  
- 각 버전별 **응답 내용·지연 시간·비용**을 비교해 보세요.

> 실습 결과는 개인별로 V0.x ~ V0.3 등의 버전을 추가하며 자유롭게 발전시키면 됩니다.

## 패키지 설치 (Colab 또는 로컬)

필요한 경우 아래 셀을 실행하여 종속성을 설치하세요.

In [ ]:
!pip install -q langfuse openai aiohttp nest_asyncio pandas python-dotenv openpyxl nbformat

## 환경 설정 및 공통 함수

In [9]:
import os, asyncio, time
import nest_asyncio, pandas as pd
from dotenv import load_dotenv
from langfuse import Langfuse

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
USE_STUB = OPENAI_API_KEY is None
langfuse = Langfuse()

if USE_STUB:
    print("🔧  Stub 모드: OPENAI_API_KEY 가 설정되지 않아 실제 API 호출 대신 더미 응답을 사용합니다.")
else:
    from langfuse.openai import AsyncOpenAI
    client = AsyncOpenAI(api_key=OPENAI_API_KEY)

nest_asyncio.apply()

# 가격(USD / token)
PRICING = {"input": 0.15/1_000_000, "output": 0.60/1_000_000}

async def call_openai(system_prompt: str, user_prompt: str, model: str = "gpt-4o-mini"):
    start = time.perf_counter_ns()
    if USE_STUB:
        await asyncio.sleep(0.05)  # 지연 시간 시뮬레이션
        answer = f"[STUB] '{user_prompt[:25]}...' 에 대한 예시 응답"
        prompt_tokens = len(system_prompt.split()) + len(user_prompt.split())
        completion_tokens = 120
    else:
        resp = await client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
        )
        answer = resp.choices[0].message.content.strip()
        usage = resp.usage
        prompt_tokens = usage.prompt_tokens
        completion_tokens = usage.completion_tokens
    latency_ms = (time.perf_counter_ns() - start) / 1_000_000
    cost = prompt_tokens * PRICING["input"] + completion_tokens * PRICING["output"]
    return {
        "answer": answer,
        "latency_ms": latency_ms,
        "prompt_tokens": prompt_tokens,
        "completion_tokens": completion_tokens,
        "usd_cost": cost,
    }

async def run_version(df: pd.DataFrame, version_name: str, build_system_prompt):
    tasks = []
    for _, row in df.iterrows():
        tasks.append(call_openai(build_system_prompt(row), row['question']))
    results = await asyncio.gather(*tasks)
    # 결과를 컬럼으로 병합
    for idx, res in enumerate(results):
        for key, val in res.items():
            df.loc[idx, f"{version_name}_{key}"] = val
    return df

## 예시 시나리오

In [10]:
scenarios = [
    {"scenario": "order_delivery", "question": "주문한 상품이 배송 예정일을 지났는데 아직 도착하지 않았어요. 어떻게 확인하나요?"},
    {"scenario": "refund", "question": "반품 신청을 했는데 환불 처리가 언제 완료되나요?"},
    {"scenario": "account_login", "question": "로그인 시도 시 2단계 인증 오류가 발생합니다. 어떻게 해결할 수 있나요?"},
]
df = pd.DataFrame(scenarios)
df

,scenario,question
0,order_delivery,주문한 상품이 배송 예정일을 지났는데 아직 도착하지 않았어요. 어떻게 확인하나요?
1,refund,반품 신청을 했는데 환불 처리가 언제 완료되나요?
2,account_login,로그인 시도 시 2단계 인증 오류가 발생합니다. 어떻게 해결할 수 있나요?


## V0.0 — Zero‑Shot (System Prompt 없이 바로 질문)

In [11]:
def sys_prompt_v0_0(row):
    return ""

await run_version(df, "V0_0", sys_prompt_v0_0)
df[['scenario', 'V0_0_answer']].head()

,scenario,V0_0_answer
0,order_delivery,주문한 상품이 배송 예정일을 지났는데 도착하지 않았다면 다음과 같은 방법으로 확인할...
1,refund,반품 신청 후 환불 처리 완료 시간은 여러 요인에 따라 다를 수 있습니다. 일반적으...
2,account_login,"2단계 인증 오류가 발생할 경우, 다음의 몇 가지 방법을 시도해 보세요:\n\n1...."


## V0.1 — Persona + Tone + Clear Instruction

In [12]:
def sys_prompt_v0_1(row):
    return (
        "You are a calm and professional Korean CS chatbot for an e‑commerce platform. "
        "Answer politely in Korean, maximum 5 sentences."
    )

await run_version(df, "V0_1", sys_prompt_v0_1)
df[['scenario','V0_1_answer']].head()

,scenario,V0_1_answer
0,order_delivery,"안녕하세요, 고객님. 배송 예정일이 지나도 상품이 도착하지 않은 점 사과드립니다. ..."
1,refund,"고객님, 반품 신청 후 환불 처리에는 보통 3-7일 정도 소요됩니다. 반품 상품이 ..."
2,account_login,"안녕하세요. 2단계 인증 오류로 인해 불편을 드려 죄송합니다. 먼저, 인증 코드가 ..."


## V0.2 — Few‑Shot + Chain‑of‑Thought

In [13]:
few_shot_examples = """
<example>
[고객] 주문한 상품이 아직 도착하지 않았어요!
[챗봇] 불편을 드려 죄송합니다. 운송장 번호 123‑4567을 조회해 보니 현재 물류센터에 있습니다. 1~2일 내 도착 예정이며, 지연 시 바로 안내드리겠습니다.
</example>
"""

def sys_prompt_v0_2(row):
    return (
        f"{few_shot_examples}\n\n"
        "You are a CS assistant. Think step‑by‑step to figure out the cause internally, "
        "but provide only the final concise answer in Korean (max 5 lines)."
    )

await run_version(df, "V0_2", sys_prompt_v0_2)
df[['scenario','V0_2_answer']].head()

,scenario,V0_2_answer
0,order_delivery,불편을 드려 죄송합니다. 주문하신 상품의 운송장 번호를 확인해 주시면 배송 상태를 ...
1,refund,반품 신청 후 환불 처리는 일반적으로 3~5일 소요됩니다. 반품 상품이 도착하고 검...
2,account_login,2단계 인증 오류는 주로 입력한 전화번호와 인증 방법 설정의 불일치 혹은 일시적인 ...


## 버전별 비교 (Latency & Cost)

In [14]:
compare_cols = ['scenario']
for v in ['V0_0','V0_1','V0_2']:
    compare_cols += [f"{v}_latency_ms", f"{v}_usd_cost"]
df[compare_cols]

,scenario,V0_0_latency_ms,V0_0_usd_cost,V0_1_latency_ms,V0_1_usd_cost,V0_2_latency_ms,V0_2_usd_cost
0,order_delivery,5201.338833,0.000153,1545.340375,0.000049,1634.396875,0.000072
1,refund,8404.851375,0.000107,1648.630500,0.000058,1440.019500,0.000050
2,account_login,5667.732541,0.000199,2128.210834,0.000068,3735.301250,0.000066


---

## ✍️ 개인 실습 영역

아래 빈 셀을 복사하여 **V0.1 ~ V0.3** 등 자신만의 변형을 시도해 보세요.  
- 새로운 System Prompt를 설계하거나  
- Few‑Shot 예시 개수를 늘리거나  
- ELI5, JSON 포맷 등 추가 요구사항을 넣어볼 수 있습니다.

In [ ]:
# TODO: 여기에 개인 실습 코드를 작성하세요.

# 끝 🎉